In [76]:
import pandas as pd
import numpy as np
import pickle as p

In [77]:
data_all = pd.read_csv('data_all.csv')

In [78]:
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=10, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

In [79]:
item_subset

[31, 2259, 1203, 5073, 102, 5501, 3818, 4270, 3767, 197]

### `/train`

In [80]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [81]:
# Model Training Result
r.json()['result']

'Success'

In [82]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict(json.loads(r.json()['cv_acc']))
cv_accuracies

,item_id,avg_sales,r2_score,mae_score,mpe_score,rmse_score
0,31.0,45.218984,0.713719,12.229731,0.270456,17.596888
1,102.0,176.608918,0.672367,37.015321,0.209589,49.535901
2,197.0,55.278581,-0.001572,24.169867,0.437237,32.251344
3,1203.0,198.355324,0.775021,30.678192,0.154663,41.541416
4,2259.0,97.155863,0.690850,21.691287,0.223263,29.162856
5,3767.0,286.172376,0.813317,40.135219,0.140248,54.825236
6,3818.0,78.448462,0.665689,16.424223,0.209363,22.196575
7,4270.0,45.090722,0.056835,30.717721,0.681243,37.535005
8,5073.0,115.369817,0.620662,28.096428,0.243534,37.605619
9,5501.0,464.741615,0.552072,120.373960,0.259013,155.600297


### `/optimize`

In [93]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [94]:
best_prices = r.json()['result']
best_prices

{'2256': 8.81,
 '2257': 11.99,
 '2721': 12.56,
 '3718': 7.42,
 '3722': 10.19,
 '3727': 10.47,
 '3732': 10.56,
 '7009': 6.09,
 '7115': 4.92,
 '7157': 7.12}

In [95]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['pred_q']

In [96]:
qty

{'2256': 79,
 '2257': 102,
 '2721': 140,
 '3718': 486,
 '3722': 1468,
 '3727': 1500,
 '3732': 424,
 '7009': 14902,
 '7115': 100,
 '7157': 27}

In [97]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (best_prices_input[item]*qty[item])
    
# Number is obscenely high due to some outliers,
# Notably 7009, likely due to lack of constraints etc.
print(revenue)

188301.96


### `/predict/`

In [69]:
# Craft mock data input, average historical prices of items
prices_input = {}
for item in item_subset:
    avg_item_price = data_subset.query('Item_ID=={}'.format(item))['Price_'].mean()
    prices_input[str(item)] = round(avg_item_price,2)
    
prices_input

{'3722': 10.72,
 '3718': 7.6,
 '2721': 13.07,
 '2256': 8.27,
 '7009': 4.62,
 '2257': 11.54,
 '3732': 11.79,
 '3727': 10.81,
 '7115': 4.81,
 '7157': 6.66}

In [70]:
import requests
import json

url = 'http://localhost:5000/predict/'



payload = {'prices': prices_input,
           'project_id': 1
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [71]:
# What If Prediction
qty = r.json()['pred_q']
qty

{'2256': 57,
 '2257': 18,
 '2721': 26,
 '3718': 291,
 '3722': 361,
 '3727': 288,
 '3732': 150,
 '7009': 1922,
 '7115': 34,
 '7157': 12}

In [72]:
# Calculate Revenue
revenue = 0
for item in qty.keys():
    revenue += (prices_input[item]*qty[item])
    
print(revenue)

21105.329999999998
